This notebook implements a Gurobi model to solve the knapsack problem. The following code block imports libraries that are used in the implementation.

In [1]:
import itertools
import json

import gurobipy
import numpy as np

#### Formulation

<u>Set(s)</u></br>
$I$ - set of items, $i \in I$</br>

<u>Parameters</u></br>
$v_{i}$ - value of item $i$</br>
$w_{i}$ - weight of item $i$</br>
$b$ - knapsack capacity</br>

<u>Decision Variables</u></br>
$X_{i}$ - 1, if item $i$ is included; 0, otherwise.</br>

\begin{align}
\text{Maximize } \sum_{i\in I} v_{i}X_{i} \hspace{15cm}~\\
\end{align}
\begin{align}
\text{subject }to\qquad\qquad\\
\sum_{i \in I} w_{i}X_{i} &\leq b. && \text{(Total weight of included items must be less than knapsack capacity)}\\
X_{i} & \in \{0, 1\}, \forall i \in I. && \text{(Assignment variables are binary)}
\end{align}

The following code block generates data for an example instance of the assignment problem

In [2]:
NUM_ITEMS = 10

I = {i for i in range(1, NUM_ITEMS+1)}

np.random.seed(0)

v = {i: round(np.random.uniform(low=1.0, high=10.0), 2) for i in I}
w = {i: round(np.random.uniform(low=1.0, high=10.0), 2) for i in I}
b = round(sum(w.values())*0.45, 2)

First, we need to instantiate a Gurobi `Model` instance

In [3]:
model = gurobipy.Model('Knapsack-Problem')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-30


Next, we define the decision variables

In [4]:
X = model.addVars(
    I,
    vtype=gurobipy.GRB.BINARY,
    name='X',
)

Next we define the objective function and specify that the defined objective should be used by our instantiated model. As a reminder, our objective is:

$$\text{Maximize } \sum_{i\in I} v_{i}X_{i}.$$

Note that we use a `LinExpr` object, along with `for` loops, to capture the sums in the expression.

In [5]:
total_value = gurobipy.LinExpr()
for i in I:
    total_value.add(v[i]*X[i])

model.setObjective(
    total_value, 
    sense=gurobipy.GRB.MAXIMIZE,
)

The following code block defines the capacity constraint. In the formulation, we represented this constraint as:

$$\sum_{i \in I} X_{i} \leq b.$$

In [6]:
i_sum = gurobipy.LinExpr()
for i in I:
    i_sum.add(w[i]*X[i])

model.addConstr(
    i_sum <= b, 
    name=f'capacity_constraint',
)

<gurobi.Constr *Awaiting Model Update*>

Before we solve the model, lets `update` our model and look at the information it contains.

In [7]:
model.update()
model

<gurobi.Model MIP instance Knapsack-Problem: 1 constrs, 10 vars, Parameter changes: Username=(user-defined)>

Let's look at the constraints

In [8]:
model.getConstrs()

[<gurobi.Constr capacity_constraint>]

Let's look at the objective

In [9]:
model.getObjective()

<gurobi.LinExpr: 5.94 X[1] + 7.44 X[2] + 6.42 X[3] + 5.9 X[4] + 4.81 X[5] + 6.81 X[6] + 4.94 X[7] + 9.03 X[8] + 9.67 X[9] + 4.45 X[10]>

The following code block solves the defined optimization model

In [10]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Pop!_OS 22.04 LTS")

CPU model: AMD Ryzen 7 5800X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0xa1a7082a
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective 36.3600000
Presolve time: 0.00s
Presolved: 1 rows, 10 columns, 10 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Found heuristic solution: objective 40.0900000

Root relaxation: objective 4.247664e+01, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

When the model is solved, you can use the `getJSONSolution` method to get information regarding the solution.

In [11]:
json.loads(model.getJSONSolution())

{'SolutionInfo': {'Status': 2,
  'Runtime': 0.007565021514892578,
  'Work': 4.844883181818176e-05,
  'ObjVal': 40.09,
  'ObjBound': 40.09,
  'ObjBoundC': 40.09,
  'MIPGap': 0,
  'IntVio': 0,
  'BoundVio': 0,
  'ConstrVio': 0,
  'IterCount': 2,
  'BarIterCount': 0,
  'NodeCount': 1,
  'SolCount': 2,
  'PoolObjBound': 40.09,
  'PoolObjVal': [40.09, 36.36]},
 'Vars': [{'VarName': 'X[2]', 'X': 1},
  {'VarName': 'X[3]', 'X': 1},
  {'VarName': 'X[5]', 'X': 1},
  {'VarName': 'X[6]', 'X': 1},
  {'VarName': 'X[7]', 'X': 1},
  {'VarName': 'X[9]', 'X': 1}]}

You can get the value of the objective by calling the `ObjVal` attribute 

In [12]:
model.ObjVal

40.09

If you want to access the specific value of a decision variable, you can use the variables `x` attribute

In [13]:
X[1].x

0.0

The following code block prints the optimal assignments for the sample problem

In [14]:
print('Optimal Assignments')
print('-'*20)
for i in I:
    if X[i].x > 0.1:
        print(f' - Item {i} included')

Optimal Assignments
--------------------
 - Item 2 included
 - Item 3 included
 - Item 5 included
 - Item 6 included
 - Item 7 included
 - Item 9 included


#### All together

The following code block includes all of the code to generate data and solve the model.

In [15]:
NUM_ITEMS = 10

I = {i for i in range(1, NUM_ITEMS+1)}

np.random.seed(0)

v = {i: round(np.random.uniform(low=1.0, high=10.0), 2) for i in I}
w = {i: round(np.random.uniform(low=1.0, high=10.0), 2) for i in I}
b = round(sum(w.values())*0.45, 2)

model = gurobipy.Model('Knapsack-Problem')

X = model.addVars(
    I,
    vtype=gurobipy.GRB.BINARY,
    name='X',
)

total_value = gurobipy.LinExpr()
for i in I:
    total_value.add(v[i]*X[i])

model.setObjective(
    total_value, 
    sense=gurobipy.GRB.MAXIMIZE,
)

i_sum = gurobipy.LinExpr()
for i in I:
    i_sum.add(w[i]*X[i])

model.addConstr(
    i_sum <= b, 
    name=f'capacity_constraint',
)

model.optimize()

print('Optimal Assignments')
print('-'*20)
for i in I:
    if X[i].x > 0.1:
        print(f' - Item {i} included')

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Pop!_OS 22.04 LTS")

CPU model: AMD Ryzen 7 5800X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0xa1a7082a
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective 36.3600000
Presolve time: 0.00s
Presolved: 1 rows, 10 columns, 10 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Found heuristic solution: objective 40.0900000

Root relaxation: objective 4.247664e+01, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    